In [1]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y

--2021-12-22 11:45:23--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   155MB/s    in 0.5s    

2021-12-22 11:45:23 (155 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=p

In [2]:
# 导入包
import torch
import torch.optim as optim
from torch import nn as nn
import torch.nn.functional as F
from torch import sigmoid
from torch.autograd import Variable
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# PART 1: find best number of epochs(可删除）

In [4]:
# read and split data
data = pd.read_csv('molnet_bace.csv')
data_sub = data[['mol', 'pIC50']]
train_data_x, test_data_x, train_y, test_y = train_test_split(data_sub['mol'], data_sub['pIC50'], test_size = 0.2, random_state = 1 )

# process train data
train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
train_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)
train_x_tensor = torch.from_numpy(train_morgan_fps_array)
train_y_tensor = torch.from_numpy(np.array(train_y))
# process test data
test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)
test_x_tensor = torch.from_numpy(test_morgan_fps_array)
test_y_tensor = torch.from_numpy(np.array(test_y))

In [5]:
class QSAR_mlp(nn.Module):
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2048, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = self.fc4(h3)
        return output

In [6]:
# test which number of spochs is best
# epochs_list = [25, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
epochs_list = [50, 80, 100, 130, 150, 170, 190, 200, 210, 230, 260, 300]
result_list = []

for epochs in epochs_list:
  print(epochs)
  #print("="*30)
  criterion = nn.MSELoss()
  model = QSAR_mlp()
  #losses = []
  optimizer = optim.Adam(model.parameters(), lr=0.005)
  for epoch in range(epochs):
    data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).float()
    optimizer.zero_grad()
    y_pred = model(data)
    #loss = F.cross_entropy(y_pred, target)
    loss = criterion(y_pred.reshape(-1), target)
    #losses.append(loss.item())
    #if epoch % 20 == 0:
      #print("Loss {}: {}".format(epoch, loss.item()))
    loss.backward()
    optimizer.step()
  # model predict
  pred_train_y = model(Variable(train_x_tensor).float()).reshape(-1).tolist()
  pred_test_y = model(Variable(test_x_tensor).float()).reshape(-1).tolist()
  nn_test_mae = round(mean_absolute_error(test_y, pred_test_y), 2)
  nn_test_mse = round(mean_squared_error(test_y, pred_test_y), 2)
  nn_test_mape = round(mean_absolute_percentage_error(test_y, pred_test_y), 2)
  nn_test_r2 = round(r2_score(test_y, pred_test_y), 2)
  nn_train_mae = round(mean_absolute_error(train_y, pred_train_y), 2)
  nn_train_mse = round(mean_squared_error(train_y, pred_train_y), 2)
  nn_train_mape = round(mean_absolute_percentage_error(train_y, pred_train_y), 2)
  nn_train_r2 = round(r2_score(train_y, pred_train_y), 2)

  nn_perf = {'mae':[nn_train_mae, nn_test_mae],
            'mse':[nn_train_mse, nn_test_mse],
            'mape':[nn_train_mape, nn_test_mape],
            'r2':[nn_train_r2, nn_test_r2]}

  result = pd.DataFrame(nn_perf, index=['train', 'test'])
  #print(result)
  result_list.append(result)

final_result = pd.concat(result_list, keys=epochs_list)



50
80
100
130
150
170
190
200
210
230
260
300


In [7]:
final_result

mae   mse  mape    r2
50  train  0.49  0.38  0.08  0.79
    test   0.61  0.56  0.10  0.67
80  train  0.34  0.21  0.05  0.89
    test   0.54  0.49  0.09  0.71
100 train  0.30  0.16  0.05  0.91
    test   0.54  0.51  0.09  0.70
130 train  0.29  0.16  0.05  0.91
    test   0.54  0.51  0.09  0.70
150 train  0.28  0.16  0.05  0.91
    test   0.52  0.48  0.09  0.71
170 train  0.19  0.09  0.03  0.95
    test   0.53  0.53  0.09  0.69
190 train  0.28  0.15  0.05  0.92
    test   0.53  0.50  0.09  0.70
200 train  0.12  0.05  0.02  0.97
    test   0.56  0.59  0.09  0.65
210 train  0.18  0.08  0.03  0.96
    test   0.52  0.51  0.09  0.70
230 train  0.10  0.04  0.02  0.98
    test   0.55  0.58  0.09  0.66
260 train  0.18  0.08  0.03  0.96
    test   0.53  0.52  0.09  0.69
300 train  0.06  0.02  0.01  0.99
    test   0.56  0.56  0.09  0.67

# PART 2: test different sets of train/test data（可删除）

In [ ]:
! ls

In [8]:
class QSAR_mlp(nn.Module):
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2048, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = self.fc4(h3)
        return output

def torch_mlp_reg(train_file, test_file):
  # process train data
  train_data = pd.read_csv(train_file)
  train_data_x = train_data['mol']
  train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
  train_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
  train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)
  train_x_tensor = torch.from_numpy(train_morgan_fps_array)
  train_y = train_data['pIC50']
  train_y_tensor = torch.from_numpy(np.array(train_y))

  # process test data
  test_data = pd.read_csv(test_file)
  test_data_x = test_data['mol']
  test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
  test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
  test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)
  test_x_tensor = torch.from_numpy(test_morgan_fps_array)
  test_y = test_data['pIC50']
  test_y_tensor = torch.from_numpy(np.array(test_y))
  # model training
  epochs = 130
  criterion = nn.MSELoss()
  model = QSAR_mlp()
  #losses = []
  optimizer = optim.Adam(model.parameters(), lr=0.005)
  for epoch in range(epochs):
    data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).float()
    optimizer.zero_grad()
    y_pred = model(data)
    #loss = F.cross_entropy(y_pred, target)
    loss = criterion(y_pred.reshape(-1), target)
    #losses.append(loss.item())
    #if epoch % 20 == 0:
      #print("Loss {}: {}".format(epoch, loss.item()))
    loss.backward()
    optimizer.step()
  # model predict
  pred_train_y = model(Variable(train_x_tensor).float()).reshape(-1).tolist()
  pred_test_y = model(Variable(test_x_tensor).float()).reshape(-1).tolist()
  # moldel evaluation
  nn_test_mae = round(mean_absolute_error(test_y, pred_test_y), 2)
  nn_test_mse = round(mean_squared_error(test_y, pred_test_y), 2)
  nn_test_mape = round(mean_absolute_percentage_error(test_y, pred_test_y), 2)
  nn_test_r2 = round(r2_score(test_y, pred_test_y), 2)
  nn_train_mae = round(mean_absolute_error(train_y, pred_train_y), 2)
  nn_train_mse = round(mean_squared_error(train_y, pred_train_y), 2)
  nn_train_mape = round(mean_absolute_percentage_error(train_y, pred_train_y), 2)
  nn_train_r2 = round(r2_score(train_y, pred_train_y), 2)

  nn_perf = {'mae':[nn_train_mae, nn_test_mae],
            'mse':[nn_train_mse, nn_test_mse],
            'mape':[nn_train_mape, nn_test_mape],
            'r2':[nn_train_r2, nn_test_r2]}

  result = pd.DataFrame(nn_perf, index=['train', 'test'])
  #print(result)
  return result


In [10]:
# molnet_bace
molnet_bace1_random = torch_mlp_reg('molnet_bace1_train_RandomSplitter.csv', 'molnet_bace1_test_RandomSplitter.csv')
molnet_bace1_scaffold = torch_mlp_reg('molnet_bace1_train_ScaffoldSplitter.csv', 'molnet_bace1_test_ScaffoldSplitter.csv')
molnet_bace1_butina = torch_mlp_reg('molnet_bace1_train_ButinaSplitter.csv', 'molnet_bace1_test_ButinaSplitter.csv')
df = pd.concat([molnet_bace1_random, molnet_bace1_scaffold, molnet_bace1_butina], keys = ['molnet_bace_random', 'molnet_bace_scaffold', 'molnet_bace_butina'])



In [11]:
df

mae   mse  mape    r2
molnet_bace_random   train  0.28  0.16  0.04  0.91
                     test   0.55  0.55  0.09  0.71
molnet_bace_scaffold train  0.26  0.14  0.04  0.92
                     test   0.79  1.13  0.12  0.27
molnet_bace_butina   train  0.24  0.13  0.04  0.92
                     test   0.93  1.60  0.16  0.31

# PART 3

In [36]:
from rdkit.Chem import MACCSkeys

class QSAR_mlp(nn.Module):
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2215, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = self.fc4(h3)
        return output

def torch_mlp_reg(train_file, test_file):
  # process train data
  train_data = pd.read_csv(train_file)
  train_data_x = train_data['mol']
  train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
  train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
  train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)

  train_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in train_mols]
  train_maccs_fps_array = np.asarray(train_maccs_fps, dtype=float)

  train_fps_array = np.concatenate([train_morgan_fps_array, train_maccs_fps_array],axis=1)



  train_x_tensor = torch.from_numpy(train_fps_array)
  train_y = train_data['pIC50']
  train_y_tensor = torch.from_numpy(np.array(train_y))

  # process test data
  test_data = pd.read_csv(test_file)
  test_data_x = test_data['mol']
  test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
  test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
  test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)

  test_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in test_mols]
  test_maccs_fps_array = np.asarray(test_maccs_fps, dtype=float)

  test_fps_array = np.concatenate([test_morgan_fps_array, test_maccs_fps_array],axis=1)


  test_x_tensor = torch.from_numpy(test_fps_array)
  test_y = test_data['pIC50']
  test_y_tensor = torch.from_numpy(np.array(test_y))
  # model training
  epochs_list = [50, 80, 100, 130, 150, 170, 190, 200, 210, 230, 250]
  res_list = []
  for epochs in epochs_list:   
    criterion = nn.MSELoss()
    model = QSAR_mlp()
    #losses = []
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    for epoch in range(epochs):
      data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).float()
      optimizer.zero_grad()
      y_pred = model(data)
      #loss = F.cross_entropy(y_pred, target)
      loss = criterion(y_pred.reshape(-1), target)
      #losses.append(loss.item())
      #if epoch % 20 == 0:
        #print("Loss {}: {}".format(epoch, loss.item()))
      loss.backward()
      optimizer.step()
    # model predict
    pred_train_y = model(Variable(train_x_tensor).float()).reshape(-1).tolist()
    pred_test_y = model(Variable(test_x_tensor).float()).reshape(-1).tolist()
    # moldel evaluation
    nn_test_mae = round(mean_absolute_error(test_y, pred_test_y), 2)
    nn_test_mse = round(mean_squared_error(test_y, pred_test_y), 2)
    nn_test_mape = round(mean_absolute_percentage_error(test_y, pred_test_y), 2)
    nn_test_r2 = round(r2_score(test_y, pred_test_y), 2)
    nn_train_mae = round(mean_absolute_error(train_y, pred_train_y), 2)
    nn_train_mse = round(mean_squared_error(train_y, pred_train_y), 2)
    nn_train_mape = round(mean_absolute_percentage_error(train_y, pred_train_y), 2)
    nn_train_r2 = round(r2_score(train_y, pred_train_y), 2)

    nn_perf = {'mae':[nn_train_mae, nn_test_mae],
            'mse':[nn_train_mse, nn_test_mse],
            'mape':[nn_train_mape, nn_test_mape],
            'r2':[nn_train_r2, nn_test_r2]}

    result = pd.DataFrame(nn_perf, index=['train', 'test'])
    res_list.append(result)
  final_res = pd.concat(res_list, keys=epochs_list)
    
  return final_res

In [37]:
# molnet_bace
molnet_bace1_random = torch_mlp_reg('molnet_bace1_train_RandomSplitter.csv', 'molnet_bace1_test_RandomSplitter.csv')
molnet_bace1_scaffold = torch_mlp_reg('molnet_bace1_train_ScaffoldSplitter.csv', 'molnet_bace1_test_ScaffoldSplitter.csv')
molnet_bace1_butina = torch_mlp_reg('molnet_bace1_train_ButinaSplitter.csv', 'molnet_bace1_test_ButinaSplitter.csv')
molnet_bace1_df = pd.concat([molnet_bace1_random, molnet_bace1_scaffold, molnet_bace1_butina], keys = ['molnet_bace_random', 'molnet_bace_scaffold', 'molnet_bace_butina'])


In [43]:
from rdkit.Chem import MACCSkeys

class QSAR_mlp(nn.Module):
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2215, 524) 
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = self.fc4(h3)
        return output

def torch_mlp_reg(train_file, test_file):
  # process train data
  train_data = pd.read_csv(train_file)
  train_data_x = train_data['canonical_smiles']
  train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
  train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
  train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)
  train_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in train_mols]
  train_maccs_fps_array = np.asarray(train_maccs_fps, dtype=float)
  train_fps_array = np.concatenate([train_morgan_fps_array, train_maccs_fps_array],axis=1)
  train_x_tensor = torch.from_numpy(train_fps_array)
  train_y = train_data['pIC50']
  train_y_tensor = torch.from_numpy(np.array(train_y))
  # process test data
  test_data = pd.read_csv(test_file)
  test_data_x = test_data['canonical_smiles']
  test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
  test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
  test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)
  test_maccs_fps = [MACCSkeys.GenMACCSKeys(mol) for mol in test_mols]
  test_maccs_fps_array = np.asarray(test_maccs_fps, dtype=float)
  test_fps_array = np.concatenate([test_morgan_fps_array, test_maccs_fps_array],axis=1)
  test_x_tensor = torch.from_numpy(test_fps_array)
  test_y = test_data['pIC50']
  test_y_tensor = torch.from_numpy(np.array(test_y))
  # model training
  epochs_list = [50, 80, 100, 130, 150, 170, 190, 200, 210, 230, 250]
  res_list = []
  for epochs in epochs_list:   
    criterion = nn.MSELoss()
    model = QSAR_mlp()
    #losses = []
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    for epoch in range(epochs):
      data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).float()
      optimizer.zero_grad()
      y_pred = model(data)
      #loss = F.cross_entropy(y_pred, target)
      loss = criterion(y_pred.reshape(-1), target)
      #losses.append(loss.item())
      #if epoch % 20 == 0:
        #print("Loss {}: {}".format(epoch, loss.item()))
      loss.backward()
      optimizer.step()
    # model predict
    pred_train_y = model(Variable(train_x_tensor).float()).reshape(-1).tolist()
    pred_test_y = model(Variable(test_x_tensor).float()).reshape(-1).tolist()
    # moldel evaluation
    nn_test_mae = round(mean_absolute_error(test_y, pred_test_y), 2)
    nn_test_mse = round(mean_squared_error(test_y, pred_test_y), 2)
    nn_test_mape = round(mean_absolute_percentage_error(test_y, pred_test_y), 2)
    nn_test_r2 = round(r2_score(test_y, pred_test_y), 2)
    nn_train_mae = round(mean_absolute_error(train_y, pred_train_y), 2)
    nn_train_mse = round(mean_squared_error(train_y, pred_train_y), 2)
    nn_train_mape = round(mean_absolute_percentage_error(train_y, pred_train_y), 2)
    nn_train_r2 = round(r2_score(train_y, pred_train_y), 2)

    nn_perf = {'mae':[nn_train_mae, nn_test_mae],
            'mse':[nn_train_mse, nn_test_mse],
            'mape':[nn_train_mape, nn_test_mape],
            'r2':[nn_train_r2, nn_test_r2]}

    result = pd.DataFrame(nn_perf, index=['train', 'test'])
    res_list.append(result)
  final_res = pd.concat(res_list, keys=epochs_list)
    
  return final_res

In [44]:
# chembl_bace
chembl_bace1_random = torch_mlp_reg('chembl_bace1_train_RandomSplitter.csv', 'chembl_bace1_test_RandomSplitter.csv')
chembl_bace1_scaffold = torch_mlp_reg('chembl_bace1_train_ScaffoldSplitter.csv', 'chembl_bace1_test_ScaffoldSplitter.csv')
chembl_bace1_butina = torch_mlp_reg('chembl_bace1_train_ButinaSplitter.csv', 'chembl_bace1_test_ButinaSplitter.csv')
chembl_bace1_df = pd.concat([chembl_bace1_random, chembl_bace1_scaffold, chembl_bace1_butina], keys = ['chembl_bace_random', 'chembl_bace_scaffold', 'chembl_bace_butina'])


In [45]:
# chembl_cdk2
chembl_cdk2_random = torch_mlp_reg('chembl_cdk2_train_RandomSplitter.csv', 'chembl_cdk2_test_RandomSplitter.csv')
chembl_cdk2_scaffold = torch_mlp_reg('chembl_cdk2_train_ScaffoldSplitter.csv', 'chembl_cdk2_test_ScaffoldSplitter.csv')
chembl_cdk2_butina = torch_mlp_reg('chembl_cdk2_train_ButinaSplitter.csv', 'chembl_cdk2_test_ButinaSplitter.csv')
chembl_cdk2_df = pd.concat([chembl_cdk2_random, chembl_cdk2_scaffold, chembl_cdk2_butina], keys = ['chembl_cdk2_random', 'chembl_cdk2_scaffold', 'chembl_cdk2_butina'])



In [46]:
# chembl_ach
chembl_ach_random = torch_mlp_reg('chembl_ach_train_RandomSplitter.csv', 'chembl_ach_test_RandomSplitter.csv')
chembl_ach_scaffold = torch_mlp_reg('chembl_ach_train_ScaffoldSplitter.csv', 'chembl_ach_test_ScaffoldSplitter.csv')
chembl_ach_butina = torch_mlp_reg('chembl_ach_train_ButinaSplitter.csv', 'chembl_ach_test_ButinaSplitter.csv')
chembl_ach_df = pd.concat([chembl_ach_random, chembl_ach_scaffold, chembl_ach_butina], keys = ['chembl_ach_random', 'chembl_ach_scaffold', 'chembl_ach_butina'])


In [47]:
with pd.ExcelWriter('four_datasets_DL_mlp_reg.xlsx') as writer:  
    molnet_bace1_df.to_excel(writer, sheet_name='molnet_bace')
    chembl_bace1_df.to_excel(writer, sheet_name='chembl_bace')
    chembl_cdk2_df.to_excel(writer, sheet_name='chembl_cdk2')
    chembl_ach_df.to_excel(writer, sheet_name='chembl_ach')
